<a href="https://colab.research.google.com/github/antonpetkoff/identifier-suggestion/blob/master/notebooks/00-colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import utilities
import os
import shutil
from subprocess import check_output

In [2]:
!nvidia-smi

Tue Jun 23 17:12:55 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [4]:
%env WORKSPACE_DIR=/content/gdrive/My Drive/src

# TODO: how can one read an environment variable?!?
%cd '/content/gdrive/My Drive/src'

env: WORKSPACE_DIR=/content/gdrive/My Drive/src
/content/gdrive/My Drive/src


In [5]:
timestamp = check_output(['date', '-u', "+%Y-%m-%dT%H-%M-%S"]).decode('utf-8').strip()

os.environ['PROJECT_DIR'] = os.path.join(
    os.environ['WORKSPACE_DIR'],
    f'identifier-suggestion-{timestamp}',
)

In [6]:
!git clone https://github.com/antonpetkoff/identifier-suggestion.git --depth 1 "${PROJECT_DIR}"

Cloning into '/content/gdrive/My Drive/src/identifier-suggestion-2020-06-23T17-12-57'...
remote: Enumerating objects: 106, done.
remote: Counting objects: 100% (106/106), done.
remote: Compressing objects: 100% (96/96), done.
remote: Total 106 (delta 1), reused 62 (delta 1), pack-reused 0
Receiving objects: 100% (106/106), 925.64 KiB | 2.43 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [7]:
os.chdir(os.environ['PROJECT_DIR'])

In [8]:
!pwd
!ls -l

/content/gdrive/My Drive/src/identifier-suggestion-2020-06-23T17-12-57
total 69
drwx------  4 root root  4096 Jun 23 17:13 data
-rw-------  1 root root 35149 Jun 23 17:13 LICENSE
drwx------  2 root root  4096 Jun 23 17:13 notebooks
-rw-------  1 root root 10022 Jun 23 17:13 README.md
drwx------  3 root root  4096 Jun 23 17:13 reports
drwx------  2 root root  4096 Jun 23 17:13 requirements
drwx------ 13 root root  4096 Jun 23 17:13 src
drwx------  3 root root  4096 Jun 23 17:13 vscode-extension


In [9]:
%tensorflow_version 2.x

import tensorflow
print(tensorflow.__version__)

2.2.0


In [10]:
# Google Colab has standard libraries like numpy, pandas, matplotlib and TF (of course) pre-installed
!pip install -r requirements/colab.txt

In [11]:
# provide secrets to the project, e.g. access to wandb
shutil.copy(
    os.path.join(os.environ['WORKSPACE_DIR'], 'secrets/.env'),
    os.environ['PROJECT_DIR']
)

'/content/gdrive/My Drive/src/identifier-suggestion-2020-06-23T17-12-57/.env'

In [12]:
# monkey-patch (mock) os.symlink to be a noop, because wandb.save() uses it, but it is not supported by Google Colab Notebooks
os.symlink = lambda *x: print('Executing mocked noop symlink with arguments', x)

In [13]:
from argparse import Namespace
from src.pipelines.baseline import run

params = {
  'dir_data': '../data/processed/subtoken/',
  'file_checkpoint_dir': 'models/checkpoints/baseline/',
  'dir_preprocessed_data': '../data/processed/seq2seq/',
  'max_input_length': 128,
  'max_output_length': 8,
  'input_vocab_size': 5000,
  'input_embedding_dim': 50,
  'output_vocab_size': 5000,
  'output_embedding_dim': 50,
  'latent_dim': 320,
  'learning_rate': 0.0001,
  'epochs': 50,
  'batch_size': 512,
  'random_seed': 1,
  'eval_averaging': 'macro'
}

run(Namespace(**params))

Initializing logger


wandb: Wandb version 0.9.1 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Experiment parameters:  Namespace(batch_size=512, dir_data='../data/processed/subtoken/', dir_preprocessed_data='../data/processed/seq2seq/', epochs=50, eval_averaging='macro', file_checkpoint_dir='models/checkpoints/baseline/', input_embedding_dim=50, input_vocab_size=5000, latent_dim=320, learning_rate=0.0001, max_input_length=128, max_output_length=8, output_embedding_dim=50, output_vocab_size=5000, random_seed=1)
Loading preprocessed files...
Loaded input vocabulary.
Loaded output vocabulary.
Loaded preprocessed files.
Model: "encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
EncoderEmbedding (Embedding) multiple                  250000    
_________________________________________________________________
EncoderLSTM (LSTM)           multiple                  474880    
Total params: 724,880
Trainable params: 724,880
Non-trainable params: 0
____________________________________________________

wandb: Visualizing heatmap.


epoch 1 evaluation time: 15.656538963317871 sec
epoch evaluation:  {'epoch': 1, 'epoch_test_loss': <tf.Tensor: shape=(), dtype=float32, numpy=2.6153462>, 'epoch_test_sparse_categorical_accuracy': <tf.Tensor: shape=(), dtype=float32, numpy=0.1892955>, 'epoch_test_precision': <tf.Tensor: shape=(), dtype=float32, numpy=0.0038432572>, 'epoch_test_recall': <tf.Tensor: shape=(), dtype=float32, numpy=0.0037997449>, 'epoch_test_f1': <tf.Tensor: shape=(), dtype=float32, numpy=0.0028390968>}


...

epoch 50 - batch 10 - loss 0.8012212514877319 - precision 0.3431932330131531 - recall 0.3310077488422394 - f1 0.32352373003959656 - sparse_categorical_accuracy 0.40117186307907104
epoch 50 - batch 20 - loss 0.7847316861152649 - precision 0.4573734402656555 - recall 0.4335070252418518 - f1 0.4289427101612091 - sparse_categorical_accuracy 0.40087890625
epoch 50 - batch 30 - loss 0.7659597396850586 - precision 0.5266144275665283 - recall 0.49615195393562317 - f1 0.49330127239227295 - sparse_categorical_accuracy 0.40293899178504944
epoch 50 - batch 40 - loss 0.7125192284584045 - precision 0.5800135135650635 - recall 0.5482656955718994 - f1 0.5452941060066223 - sparse_categorical_accuracy 0.4038783609867096
epoch 50 - batch 50 - loss 0.7806219458580017 - precision 0.6152063608169556 - recall 0.5806769132614136 - f1 0.5786612629890442 - sparse_categorical_accuracy 0.40420758724212646
epoch 50 - batch 60 - loss 0.7839121222496033 - precision 0.6401892304420471 - recall 0.6026710271835327 - f1

wandb: Visualizing heatmap.


epoch 50 evaluation time: 17.45117163658142 sec
epoch evaluation:  {'epoch': 50, 'epoch_test_loss': <tf.Tensor: shape=(), dtype=float32, numpy=1.3682089>, 'epoch_test_sparse_categorical_accuracy': <tf.Tensor: shape=(), dtype=float32, numpy=0.33437392>, 'epoch_test_precision': <tf.Tensor: shape=(), dtype=float32, numpy=0.50616163>, 'epoch_test_recall': <tf.Tensor: shape=(), dtype=float32, numpy=0.42379072>, 'epoch_test_f1': <tf.Tensor: shape=(), dtype=float32, numpy=0.43450367>}


wandb: Visualizing heatmap.
wandb: Visualizing heatmap.
wandb: Visualizing heatmap.
wandb: Visualizing heatmap.
wandb: Visualizing heatmap.
wandb: Visualizing heatmap.
wandb: Visualizing heatmap.
wandb: Visualizing heatmap.
wandb: Visualizing heatmap.


Saving checkpoint...
Saved checkpoint: models/checkpoints/baseline/ckpt-18
Done saving model
